<a href="https://colab.research.google.com/github/MinhChanh229/SQL-data-cleaning/blob/main/Data_driven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Import pandas

In [1]:
import pandas as pd

In [12]:
google_sheet_id = '10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw'
url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?format=xlsx'

monthly_expense = pd.read_excel(url, sheet_name='Sheet1')


In [19]:
google_sheet_id = '1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4'
url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?format=xlsx'

payroll = pd.read_excel(url, sheet_name='Sheet1')


In [15]:
google_sheet_id = '1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c'
url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?format=xlsx'

daily_marketing = pd.read_excel(url, sheet_name='Sheet1')


In [56]:
google_sheet_id = '1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI'
url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?format=xlsx'

lifespan = pd.read_excel(url, sheet_name='Sheet1')

In [29]:
google_sheet_id = '1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE'
url = f'https://docs.google.com/spreadsheets/d/{google_sheet_id}/export?format=xlsx'

payments = pd.read_excel(url, sheet_name='Sheet1')

In [9]:
monthly_expense.sample(5)

,#,month,category,item,amount
24,25,2023-03-01,Office Rental,Office Rent,10000
14,15,2023-02-01,Software Licenses,Zoom,520
18,19,2023-03-01,Server Costs,AWS Hosting,8400
7,8,2023-01-01,Other,Office Supplies,500
17,18,2023-02-01,Other,Travel Expenses,3100


In [10]:
expense_202303 = monthly_expense[monthly_expense['month'] == '2023-03-01']

#1.1 Marketing Software expense

In [11]:
crm_expense = expense_202303[expense_202303['item'] == 'Salesforce']['amount'].values[0].sum()
crm_expense


np.int64(1700)

#1.2 Sales & Marketing Salaries

In [20]:
payroll.sample(5)

,month,department,employee_name,position,paid
22,2023-02-01,Marketing,Bob Davis,Content Specialist,700
11,2023-01-01,Support,Hannah Scott,Support Lead,600
37,2023-03-01,Sales,Laura Miller,Sales Associate,800
7,2023-01-01,Engineering,Diana Lee,Developer,1700
44,2023-03-01,HR,Gary Black,HR Specialist,700


In [21]:
payroll_202303 = payroll[payroll['month'] == '2023-03-01']

In [22]:
sale_marketing_salary = payroll_202303[(payroll_202303['department'] == 'Sales') |\
                                       (payroll_202303['department'] == 'Marketing')]['paid'].sum()
sale_marketing_salary

np.int64(5950)

#1.3 Daily marketing

In [16]:
daily_marketing.sample(5)

,date,channel,spending
288,2023-03-14,Google Ads,639
350,2023-03-29,LinkedIn Ads,762
128,2023-02-02,Google Ads,612
201,2023-02-20,Facebook Ads,943
0,2023-01-01,Google Ads,784


In [41]:
daily_marketing_202303 = daily_marketing[(daily_marketing['date']>='2023-03-01') & (daily_marketing['date']<='2023-03-31')]

In [42]:
daily_marketing_expense = daily_marketing_202303['spending'].sum()
daily_marketing_expense

np.int64(68830)

#1.4 Number of new customers

In [31]:
payments.sample(5)

,date,customer_id,receipt_amount,new_customer
193,2023-01-20,2820,410,1
833,2023-03-22,2516,95,0
874,2023-03-26,2714,123,0
414,2023-02-10,2575,178,1
899,2023-03-27,1816,307,1


In [38]:
payments_202303 = payments[(payments['date'] >= '2023-03-01') & (payments['date']<= '2023-03-31')]

In [39]:
number_of_new_customers = payments_202303[payments_202303['new_customer'] == 1]['customer_id'].nunique()
number_of_new_customers

63

#1.5 CAC Calculation

In [43]:
total_acquisition_cost = crm_expense + sale_marketing_salary + daily_marketing_expense
cac = round(total_acquisition_cost / number_of_new_customers, 2)
cac

np.float64(1213.97)

#2.1 ARPU

In [45]:
total_revenue = payments_202303['receipt_amount'].sum()
total_revenue

np.int64(83033)

#2.2 Number of customers

In [46]:
number_of_customers = payments_202303['customer_id'].nunique()
number_of_customers

292

#2.3 ARPU calculation

In [47]:
arpu = round(total_revenue / number_of_customers, 2)
arpu

np.float64(284.36)

#3 COGS
##3.1 Software expenses

In [48]:
production_expense = ['AWS Hosting', 'Google Cloud Storage', 'Atlassian Jira']
software_expense = expense_202303[expense_202303['item'].isin(production_expense)]['amount'].sum()
software_expense

np.int64(14200)

In [49]:
shared_software = ['Slack', 'Zoom']
shared_software_expense = expense_202303[expense_202303['item'].isin(shared_software)]['amount'].sum()*0.6
shared_software_expense

np.float64(864.0)

##3.2 Production salary

In [51]:
production_salary = payroll_202303[payroll_202303['department'] == 'Engineering']['paid'].sum()
production_salary

np.int64(5200)

##3.3 COGS calculation

In [52]:
cogs = software_expense + shared_software_expense + production_salary
cogs

np.float64(20264.0)

#4 Gross profit margin

In [55]:
gross_margin = round((total_revenue - cogs) / total_revenue * 100, 2)
gross_margin

np.float64(75.6)

#5 LTV - Life time value

In [58]:
lifespan['lifespan_days'] = pd.to_datetime(lifespan['churn_date']) - pd.to_datetime(lifespan['start_date'])
lifespan['lifespan_days'] = lifespan['lifespan_days'].dt.days
lifespan.sample(5)

,Unnamed: 0,start_date,churn_date,lifespan_days
47,1047,2022-02-21,2022-03-31,38
82,1082,2021-09-15,2021-10-06,21
4,1004,2021-08-23,2022-02-02,163
94,1094,2020-10-17,2021-05-09,204
74,1074,2023-02-27,2023-03-18,19


In [59]:
avg_lifespan_month = lifespan['lifespan_days'].mean() / 30
avg_lifespan_month

np.float64(9.841333333333333)

In [60]:
ltv = arpu * avg_lifespan_month * gross_margin
ltv

np.float64(211565.204928)

#Conclusion

1.
- Chi phí thu hút khách hàng (CAC) cao, nhưng hợp lý
Với CAC là $1,213.97, chi phí để có được một khách hàng mới khá cao.

- Tuy nhiên, nếu dịch vụ/sản phẩm bạn bán có tính lâu dài hoặc giá trị cao (như SaaS, dịch vụ tài chính, y tế, v.v.) thì mức CAC này có thể chấp nhận được.

2.
- Doanh thu trung bình trên mỗi khách hàng (ARPU) tương đối thấp
ARPU là $284.36, thấp hơn nhiều so với CAC.

- Điều này cho thấy bạn phải giữ khách hàng lâu dài hoặc tăng giá trị mỗi lần mua để bù lại chi phí ban đầu bỏ ra.



3.
- Biên lợi nhuận gộp rất cao (75.6%)
Đây là điểm sáng nhất! Cho thấy bạn có chi phí sản xuất thấp hoặc mô hình kinh doanh hiệu quả.

- Biên lợi nhuận cao giúp bạn có dư địa lớn để chi cho marketing, cải tiến sản phẩm, hoặc tăng trưởng.

4.
- LTV vượt trội so với CAC
LTV / CAC = 211,565.20 / 1,213.97 ≈ 174.3

- Đây là một tỷ lệ cực kỳ ấn tượng (tốt nếu >3, còn bạn >170).

- Nghĩa là giá trị trọn đời của khách hàng rất lớn so với chi phí để thu hút họ, cho thấy khả năng sinh lời mạnh và tiềm năng mở rộng quy mô.

 Tóm tắt đây là một mô hình kinh doanh hiệu quả cao về lợi nhuận (gross margin), và cực kỳ tối ưu trong dài hạn (LTV rất cao).

 Tuy nhiên, CAC khá cao và ARPU thấp, điều đó đòi hỏi bạn phải:

Giữ chân khách hàng lâu dài để LTV thực sự được hiện thực hóa.

Tăng ARPU thông qua bán chéo (cross-sell), bán thêm (up-sell), hoặc định giá lại dịch vụ.

Tối ưu chi phí CAC nếu muốn mở rộng quy mô nhanh hơn.

